In [3]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
# Load dataset
x = r"C:\Users\tnag3\OneDrive\Documents\Datasets\recipes_cleaned.csv"
recipe_df = pd.read_csv(x, encoding='latin1')

In [7]:
# Handle null values
recipe_df['ingredients_list'] = recipe_df['ingredients_list'].fillna('')
recipe_df['cooking_procedure'] = recipe_df['cooking_procedure'].fillna('')


In [9]:
# Convert ingredients to TF-IDF vectors
vectorizer = TfidfVectorizer()
X_ingredients = vectorizer.fit_transform(recipe_df['ingredients_list'])


In [11]:
# Train KNN model
knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(X_ingredients)


NearestNeighbors(metric='euclidean')

In [21]:
# Recommendation function
def recommend_recipes(input_ingredients):
    try:
        input_ingredients_transformed = vectorizer.transform([input_ingredients])
        distances, indices = knn.kneighbors(input_ingredients_transformed)
        recommendations = recipe_df.iloc[indices[0]]
        
        # Apply intersection: Filter recipes whose ingredients match all of the input ingredients
        recommendations = recommendations[recommendations['ingredients_list'].apply(
            lambda ingredients: all(ingredient in ingredients for ingredient in input_ingredients.split(',')))
        ]
        
        if recommendations.empty:
            raise ValueError("No recipes found for the given ingredients.")
        
        return recommendations[['recipe_name']]
    
    except Exception as e:
        return None

# Function to display recommended recipes
def show_recommendations():
    input_ingredients = entry_ingredients.get()
    
    # Check if the input ingredients field is empty
    if not input_ingredients.strip():
        error_label.config(text="Please enter some ingredients.", fg="red")
        return
    
    recommendations = recommend_recipes(input_ingredients)
    
    # If no recommendations are found, show an error once
    if recommendations is None:
        error_label.config(text="No recipes found for the given ingredients.", fg="red")
        return
    
    # Clear the listbox and show the new recommendations
    recipe_listbox.delete(0, tk.END)
    for _, row in recommendations.iterrows():
        recipe_listbox.insert(tk.END, row['recipe_name'])
    
    # Clear the error label if recommendations are found
    error_label.config(text="")

# Function to display recipe details
def show_recipe_details():
    try:
        selected = recipe_listbox.get(recipe_listbox.curselection())
        
        if not selected:
            error_label.config(text="No recipe selected.", fg="red")
            return
        
        recipe = recipe_df[recipe_df['recipe_name'] == selected]
        
        if recipe.empty:
            error_label.config(text="Recipe details not found.", fg="red")
            return
        
        details = recipe[['ingredients_list', 'cooking_procedure']].iloc[0]
        
        details_text.set(
            f"Ingredients: {details['ingredients_list']}\n\nProcedure: {details['cooking_procedure']}"
        )
        
        # Clear the error label if details are fetched
        error_label.config(text="")
    
    except Exception as e:
        error_label.config(text=f"An error occurred while fetching recipe details: {e}", fg="red")








In [23]:
# Import necessary libraries
import tkinter as tk
from tkinter import messagebox

# Initialize Tkinter application
root = tk.Tk()
root.title("Recipe Recommendation System")

# Input for ingredients
tk.Label(root, text="Enter ingredients:").pack()
entry_ingredients = tk.Entry(root, width=70)
entry_ingredients.pack()

# Button to fetch recommendations
btn_recommend = tk.Button(root, text="Get Recommendations", command=show_recommendations)
btn_recommend.pack()

# Listbox to display recommended recipes
recipe_listbox = tk.Listbox(root, height=15, width=150)
recipe_listbox.pack()

# Button to fetch recipe details
btn_details = tk.Button(root, text="View Recipe Details", command=show_recipe_details)
btn_details.pack()

# Label to display recipe details
details_text = tk.StringVar()
tk.Label(root, textvariable=details_text, wraplength=400, justify="left").pack()

# Error message label
error_label = tk.Label(root, text="", fg="red")
error_label.pack()

# Start the Tkinter main loop
root.mainloop()
